# 結合　Join

In [ ]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()


データの確認

[hotel.csv](/user/{username}/edit/awesomebook/data/hotel.csv)



## 内部結合　Inner join


In [ ]:
inner_join(reserve_tb %>% filter(people_num == 1),
           hotel_tb %>% filter(is_business),
           by='hotel_id')


## リコメンド　Recommend


In [ ]:
# small_area_nameごとにホテル数をカウント、結合キーを判定するためのテーブル
# A table to count the number of hotels per small_area_name and to determine the join key.
small_area_mst <-
  hotel_tb %>%
    group_by(big_area_name, small_area_name) %>%

    # -1は、自ホテルを引いている
    # -1 is subtracting its own hotel.
    summarise(hotel_cnt=n() - 1) %>%

    # 集約処理完了後に、グループ化を解除
    # Remove grouping after consolidation is complete.
    ungroup() %>%

    # 20件以上であればjoin_area_idをsmall_area_nameとして設定
    # 20件未満であればjoin_area_idをbig_area_nameとして設定
    # If it is 20 or more, join_area_id is set as small_area_name.
    # If it is less than 20, set join_area_id as big_area_name.
    mutate(join_area_id=
             if_else(hotel_cnt >= 20, small_area_name, big_area_name)) %>%
    select(small_area_name, join_area_id)

# レコメンド元になるホテルにsmall_area_mstを結合することで、join_area_idを設定
# Set join_area_id by joining small_area_mst to the hotel that is the recommendation source.
base_hotel_mst <-
  inner_join(hotel_tb, small_area_mst, by='small_area_name') %>%
    select(hotel_id, join_area_id)

# 必要に応じて、メモリを解放(必須ではないがメモリ量に余裕のないときに利用)
# Memory release
rm(small_area_mst)

# recommend_hotel_mstはレコメンド候補のためのテーブル
# recommend_hotel_mst is a table for recommendation candidates.
recommend_hotel_mst <-
  bind_rows(
    # join_area_idをbig_area_nameとしたレコメンド候補マスタ
    # It is a recommendation candidate master whose join_area_id is big_area_name.
    hotel_tb %>%
      rename(rec_hotel_id=hotel_id, join_area_id=big_area_name) %>%
      select(join_area_id, rec_hotel_id),

    # join_area_idをsmall_area_nameとしたレコメンド候補マスタ
    # It is a recommendation candidate master whose join_area_id is small_area_name.
    hotel_tb %>%
      rename(rec_hotel_id=hotel_id, join_area_id=small_area_name) %>%
      select(join_area_id, rec_hotel_id)
  )

# base_hotel_mstとrecommend_hotel_mstを結合し、レコメンド候補の情報を付与
inner_join(base_hotel_mst, recommend_hotel_mst, by='join_area_id') %>%

  # レコメンド候補から自分を除く
  filter(hotel_id != rec_hotel_id)


# 004_join03_a

In [ ]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
reserve_tb %>%

  # group_byによって、customer_idごとにデータをグループ化
  group_by(customer_id) %>%

  # LAG関数を利用し、2件前のtotal_priceをbefore_priceとして取得
  # LAG関数によって参照する際のグループ内のデータをreserve_datetimeの古い順に指定
  mutate(before_price=lag(total_price, n=2,
                          order_by=reserve_datetime, default=NA))


# 004_join03_b

In [ ]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
# roll_sum関数のためのライブラリ
library(RcppRoll)

reserve_tb %>%

  # データ行をcustomer_idごとにグループ化
  group_by(customer_id) %>%

  # customer_idごとにreserve_datetimeでデータを並び替え
  arrange(reserve_datetime) %>%

  # RcppRollのroll_sumによって、移動合計値を計算
  mutate(price_sum=roll_sum(total_price, n=3, align='right', fill=NA))


# 004_join03_c

In [ ]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
# row_number関数でreserve_datetimeを利用するために、POSIXct型に変換
# (「第10章 日時型」で詳しく解説)
reserve_tb$reserve_datetime <-
  as.POSIXct(reserve_tb$reserve_datetime, format='%Y-%m-%d %H:%M:%S')

reserve_tb %>%
  group_by(customer_id) %>%
  arrange(reserve_datetime) %>%

  # １〜3件前のtotal_priceの合計をlag関数によって計算
  # if_else関数とrank関数を組み合わせて、何件合計したかを判定
  # order_by=reserve_datetimeの指定は、事前に並び替えられているので必須ではない
  # 合計した件数が0だった場合、0で割っているためprice_avgがNANとなる
  mutate(price_avg=
           (  lag(total_price, 1, order_by=reserve_datetime, default=0)
            + lag(total_price, 2, order_by=reserve_datetime, default=0)
            + lag(total_price, 3, order_by=reserve_datetime, default=0))
           / if_else(row_number(reserve_datetime) > 3,
                     3, row_number(reserve_datetime) - 1))


# 004_join03_d

In [ ]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
library(tidyr)

# row_number関数でreserve_datetimeを利用するために、POSIXct型に変換
# (「第10章 日時型」で詳しく解説)
reserve_tb$reserve_datetime <-
  as.POSIXct(reserve_tb$reserve_datetime, format='%Y-%m-%d %H:%M:%S')

# 過去90日間の合計予約金額を計算したテーブル
sum_table <-

  # reserve_datetimeの日付を確認せずに、同じcustomer_idのデータ行同士をすべて結合
  inner_join(
    reserve_tb %>%
      select(reserve_id, customer_id, reserve_datetime),
    reserve_tb %>%
      select(customer_id, reserve_datetime, total_price) %>%
      rename(reserve_datetime_before=reserve_datetime),
    by='customer_id') %>%

  # checkinの日付を比較して、90日以内のデータが結合されているデータ行のみ抽出
  # 60*60*24*90は、60秒*60分*24時間*90日を意味し、90日間分の秒数を計算
  # (日付のデータ型については、「第10章 日時型」で詳しく解説)
  filter(reserve_datetime > reserve_datetime_before &
           reserve_datetime - 60 * 60 * 24 * 90 <= reserve_datetime_before) %>%
  select(reserve_id, total_price) %>%

  # reserve_idごとにtotal_priceの合計値を計算
  group_by(reserve_id) %>%
  summarise(total_price_90d=sum(total_price)) %>%
  select(reserve_id, total_price_90d)

# 計算した合計値を結合し、元のテーブルに情報を付与
# 合計値が存在しないレコードの合計値の値を、replace_naを利用して0に変更
left_join(reserve_tb, sum_table, by='reserve_id') %>%
  replace_na(list(total_price_90d=0))


# 004_join04

In [ ]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
library(tidyverse)

# 計算対象の年月のデータフレームを作成
month_mst <- data.frame(year_month=
  # 2017-01-01、2017-02-01, 2017-03-01を生成し、format関数で形式を年月に変換
  # (日付のデータ型については、「第10章 日時型」で詳しく解説)
  format(seq(as.Date('2017-01-01'), as.Date('2017-03-01'), by='months'),
         format='%Y%m')
)

# 顧客IDと計算対象のすべての年月が結合したテーブル
customer_mst <-

  # すべての顧客IDと年月マスタを全結合
  merge(customer_tb %>% select(customer_id), month_mst) %>%

  # mergeで指定した結合キーのデータ型がカテゴリ型になっているので、文字型に戻す
  # (カテゴリ型については、「第9章 カテゴリ型」で詳しく解説)
  mutate(customer_id=as.character(customer_id),
         year_month=as.character(year_month))

# 合計利用金額を月ごとに計算
left_join(
  customer_mst,

  # 予約テーブルに年月の結合キーを準備
  reserve_tb %>%
    mutate(checkin_month = format(as.Date(checkin_date), format='%Y%m')),

  # 同じcustomer_idと年月を結合
  by=c('customer_id'='customer_id', 'year_month'='checkin_month')
) %>%

  # customer_idと年月で集約
  group_by(customer_id, year_month) %>%

  # 合計金額を算出
  summarise(price_sum=sum(total_price)) %>%

  # 予約レコードがなかった場合の合計金額を値なしから0に変換
  replace_na(list(price_sum=0))
